In [81]:
from openpyxl import Workbook, load_workbook
from fuzzywuzzy import process, fuzz
import time
import pandas as pd
import numpy as np

In [54]:
from szp_funcs import print_df, path_to_docs, path_to_data, to_double

In [15]:
all = pd.read_excel(path_to_docs + 'pay_type.xlsx', sheet_name='Все').rename(columns={'Вид ': 'type', 'Наименование ': 'name'})[['type', 'name']]

In [23]:
look_up = all['name'].to_list()

In [20]:
apr_names = pd.read_excel(path_to_docs + 'pay_type.xlsx', sheet_name='Апрель').rename(columns={'Наименование начисления': 'name'})

In [21]:
apr_names

,name
0,Выплата за результативность работы в предыдущ...
1,Выплата за результативность работы в предыдущ...
2,Выплаты за результативность работы в предыдущ...
3,Должностной оклад
4,Должностной оклад
...,...
2291,Федеральное ежемесячное денежное вознагр-ие за...
2292,Хозяйственное обслуживание здания (в рамках ув...
2293,Часы Доп.Образ.Услуги (курсы внебюджет)
2294,Часы заочного обучения (внебюджет)


In [31]:
def matcher(row):
    ans = process.extractOne(row['name'], look_up, scorer=fuzz.WRatio, score_cutoff=60)
    if ans:
        return ans[0]
    return ans

In [32]:
apr_names['new_name'] = apr_names.apply(lambda row: matcher(row), axis=1)

In [35]:
apr_names = apr_names.rename(columns={'name': 'old_name'})

In [38]:
nach_to_type = pd.merge(apr_names, all, how='left', left_on='new_name', right_on='name').drop(columns=['name', 'new_name'])

In [132]:
nach_to_type

,old_name,type
0,Выплата за результативность работы в предыдущ...,Стимулирующие выплаты
1,Выплата за результативность работы в предыдущ...,Стимулирующие выплаты
2,Выплаты за результативность работы в предыдущ...,Стимулирующие выплаты
3,Должностной оклад,Базовая часть
4,Должностной оклад,Базовая часть
...,...,...
2291,Федеральное ежемесячное денежное вознагр-ие за...,Компенсационные выплаты
2292,Хозяйственное обслуживание здания (в рамках ув...,Компенсационные выплаты
2293,Часы Доп.Образ.Услуги (курсы внебюджет),NaN
2294,Часы заочного обучения (внебюджет),NaN


In [40]:
nach_to_type[nach_to_type['type'].isna()]

,old_name,type
18,**Отпуск учебный дополнительный оплачиваемый,NaN
64,Вводный курс,NaN
67,ВКР внебюджет,NaN
68,Внеурочная деятельность,NaN
69,Внутреннее совместительство,NaN
...,...,...
2239,Стимулирующий коэффициент,NaN
2250,"Удлиненный рабочий день Курв (0,2)",NaN
2293,Часы Доп.Образ.Услуги (курсы внебюджет),NaN
2294,Часы заочного обучения (внебюджет),NaN


In [123]:
print_df(nach_to_type, 'виды начислений в апреле')

In [72]:
inn = 7712022159

In [42]:
df_base = pd.read_excel(path_to_data + 'apr.xlsx')

In [43]:
df_nach = pd.read_excel(path_to_data + 'apr_nach.xlsx')

In [44]:
df_gph = pd.read_excel(path_to_data + 'apr_gph.xlsx')

In [51]:
df_jobs = pd.read_excel(path_to_docs + 'input.xlsx', sheet_name='УГД').rename(columns={'Должность': 'job', 'УГД': 'grp', 'Педагогический': 'ped',
'Работники непосредственно осуществляющие и обеспечивающие основной учебно-вспомогательный процесс во взаимодействии с детьми': 'isp',
'общее образование': 'obh', 'доп обр': 'dop', 'прочий': 'other'})

In [56]:
df_gph['status_pref'] = df_gph['status']

In [90]:
df_gph['type'] = 'gph'

In [91]:
res = pd.concat([df_base, df_gph])

In [92]:
res['stv'] = res.apply(lambda row: to_double(row, 'stv'), axis=1)

In [93]:
res['spis'] = res['stv'] * res['wday'] / res['nday']

In [94]:
res['type'].unique()

array(['Основное место работы', 'Внешнее совместительство',
       'Внутреннее совместительство', nan, 'gph'], dtype=object)

In [63]:
res

,inn,snils,sum,job,day,type,stv,status,status_pref,wday,nday,spis
0,7734408766,001-879-913 57,135113.91,Преподаватель,1.0,Основное место работы,1.00,Работа,Работа,22.0,21.0,1.047619
1,7734408766,001-896-420 39,126320.51,Преподаватель,1.0,Основное место работы,1.00,Работа,Работа,23.0,21.0,1.095238
2,7734408766,002-050-383 67,68007.79,Преподаватель,1.0,Основное место работы,1.00,Работа,Работа,21.0,21.0,1.000000
3,7734408766,002-065-271 85,76705.72,Преподаватель,1.0,Основное место работы,1.00,Работа,Работа,21.0,21.0,1.000000
4,7734408766,002-845-525 25,125265.61,Преподаватель,1.0,Основное место работы,1.00,Работа,Работа,22.0,21.0,1.047619
...,...,...,...,...,...,...,...,...,...,...,...,...
7126,7722069597,115-482-161 39,55776.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7127,7722069597,147-927-788 25,30000.00,Педагог дополнительного образования,NaN,Внешнее совместительство,0.11,Увольнение,Увольнение,NaN,NaN,NaN
7128,7722069597,161-352-731 42,6000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7129,7722069597,163-990-562 04,5250.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
comp_res = pd.merge(res[['inn', 'snils', 'sum']].groupby(['inn', 'snils']).sum().reset_index(), res.loc[:, res.columns != 'sum'][res.type != 'Внутреннее совместительство'], how='left', on=['inn', 'snils']).drop_duplicates(['inn', 'snils', 'sum'])

In [64]:
df_jobs

,job,grp,ped,isp,obh,dop,other
0,Администратор,Прочий персонал (включая иной административный...,0,0,0,0,1
1,Администратор баз данных,Прочий персонал (включая иной административный...,0,0,0,0,1
2,Аккомпаниатор-концертмейстер,Прочий персонал (включая иной административный...,0,0,0,0,1
3,Аналитик,Прочий персонал (включая иной административный...,0,0,0,0,1
4,Архивариус,Прочий персонал (включая иной административный...,0,0,0,0,1
...,...,...,...,...,...,...,...
300,Экскурсовод,Прочий персонал (включая иной административный...,0,0,0,0,1
301,Эксперт,Прочий персонал (включая иной административный...,0,0,0,0,1
302,Электромеханик,Прочий персонал (включая иной административный...,0,0,0,0,1
303,Электромонтер по ремонту и обслуживанию электр...,Прочий персонал (включая иной административный...,0,0,0,0,1


In [96]:
rows = {
    'ped_all': comp_res[(comp_res.job.isin(df_jobs[df_jobs['ped'] == 1]['job'].to_list())) & (comp_res['inn'] == inn)],
    'obh': comp_res[(comp_res.job.isin(df_jobs[df_jobs['obh'] == 1]['job'].to_list())) & (comp_res['inn'] == inn)],
    'teach': comp_res[(comp_res.job.isin(['Учитель'])) & (comp_res['inn'] == inn)],
    'pre_sch': comp_res[(comp_res.job.isin(['Воспитатель'])) & (comp_res['inn'] == inn)],
    'vosp': comp_res[(comp_res.job.isin(['Воспитатель'])) & (comp_res['inn'] == inn)],
    'dop_ppl': comp_res[(comp_res.job.isin(df_jobs[df_jobs['dop'] == 1]['job'].to_list())) & (comp_res['inn'] == inn)],
    'dop_ped': comp_res[(comp_res.job.isin(['Педагог дополнительного образования'])) & (comp_res['inn'] == inn)],
    'aup': comp_res[(comp_res.job.isin(df_jobs[df_jobs['grp'] == 'Административно-управленческий персонал']['job'].to_list())) & (comp_res['inn'] == inn)],
    'ruk': comp_res[(comp_res.job.isin(['Директор'])) & (comp_res['inn'] == inn)],
    'other_aup': comp_res[(comp_res.job.isin(df_jobs[df_jobs['grp'] == 'Административно-управленческий персонал']['job'].to_list())) & (comp_res['inn'] == inn) & (comp_res.job.isin(['Директор']) == False)],
    'other': comp_res[(comp_res.job.isin(df_jobs[df_jobs['other'] == 1]['job'].to_list())) & (comp_res['inn'] == inn)] ,
    'uvp': comp_res[(comp_res.job.isin(df_jobs[df_jobs['grp'] == 'Учебно-вспомогательный персонал']['job'].to_list())) & (comp_res['inn'] == inn)],
    'obs': comp_res[(comp_res.job.isin(df_jobs[df_jobs['grp'] == 'Учебно-вспомогательный персонал']['job'].to_list()) == False) & (comp_res['inn'] == inn) & (comp_res.job.isin(df_jobs[df_jobs['other'] == 1]['job'].to_list()))],
    'all': comp_res[comp_res['inn'] == inn],
}

In [131]:
rows['all'][rows['all']['stv'].isna()]['sum'].sum()/5

14878.541999999998

In [101]:
list(rows.keys())

['ped_all',
 'obh',
 'teach',
 'pre_sch',
 'vosp',
 'dop_ppl',
 'dop_ped',
 'aup',
 'ruk',
 'other_aup',
 'other',
 'uvp',
 'obs',
 'all']

In [102]:
ans = pd.DataFrame({'group': list(rows.keys())})

In [121]:
rows['ped_all'][rows['ped_all']['type'] == 'Внешнее совместительство']

,inn,snils,sum,job,day,type,stv,status,status_pref,wday,nday,spis
21493,7712022159,042-922-901 45,42696.66,Педагог дополнительного образования,0.0,Внешнее совместительство,0.22,Работа,Работа,NaN,NaN,NaN


In [124]:
rows['all'][(rows['all']['stv'].isna()) | (rows['all']['wday'].isna()) | (rows['all']['nday'].isna())]

,inn,snils,sum,job,day,type,stv,status,status_pref,wday,nday,spis
21487,7712022159,021-126-216 77,4132.00,NaN,NaN,gph,NaN,NaN,NaN,NaN,NaN,NaN
21491,7712022159,027-553-216 45,16225.00,NaN,NaN,gph,NaN,NaN,NaN,NaN,NaN,NaN
21492,7712022159,029-907-577 96,5250.00,NaN,NaN,gph,NaN,NaN,NaN,NaN,NaN,NaN
21493,7712022159,042-922-901 45,42696.66,Педагог дополнительного образования,0.0,Внешнее совместительство,0.22,Работа,Работа,NaN,NaN,NaN
21511,7712022159,127-451-415 45,22140.00,Педагог дополнительного образования,0.0,Основное место работы,0.22,Работа,Работа,NaN,NaN,NaN
21525,7712022159,181-136-942 61,14000.00,NaN,NaN,gph,NaN,NaN,NaN,NaN,NaN,NaN
21528,7712022159,201-608-634 20,34785.71,NaN,NaN,gph,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
ans

,group,spis_all,spis_osn,spis_vnes,spis_gph,fot_all,fot_osn,fot_vnes,fot_gph,szp_osn,szp_vnes,szp_gph
0,ped_all,55.927462,55.927462,0.0,0.0,4501.21277,4458.51611,42.69666,0.00000,7.971962e+04,inf,NaN
1,obh,55.927462,55.927462,0.0,0.0,4501.21277,4458.51611,42.69666,0.00000,7.971962e+04,inf,NaN
2,teach,45.692077,45.692077,0.0,0.0,3623.80727,3623.80727,0.00000,0.00000,7.930931e+04,NaN,NaN
3,pre_sch,2.110000,2.110000,0.0,0.0,116.47800,116.47800,0.00000,0.00000,5.520284e+04,NaN,NaN
4,vosp,2.110000,2.110000,0.0,0.0,116.47800,116.47800,0.00000,0.00000,5.520284e+04,NaN,NaN
5,dop_ppl,10.235385,10.235385,0.0,0.0,877.40550,834.70884,42.69666,0.00000,8.155129e+04,inf,NaN
6,dop_ped,0.000000,0.000000,0.0,0.0,64.83666,22.14000,42.69666,0.00000,inf,inf,NaN
7,aup,2.000000,2.000000,0.0,0.0,304.04302,304.04302,0.00000,0.00000,1.520215e+05,NaN,NaN
8,ruk,0.000000,0.000000,0.0,0.0,0.00000,0.00000,0.00000,0.00000,NaN,NaN,NaN
9,other_aup,2.000000,2.000000,0.0,0.0,304.04302,304.04302,0.00000,0.00000,1.520215e+05,NaN,NaN


In [122]:
print_df(ans, 'свод, 14 строк без структуры, 1249')

In [107]:
def spis_all(row):
    i = row['group']
    return rows[i]['spis'].sum()

def spis_osn(row):
    i = i = row['group']
    return rows[i][rows[i]['type'] == 'Основное место работы']['spis'].sum()

def spis_vnes(row):
    i = i = row['group']
    return rows[i][rows[i]['type'] == 'Внешнее совместительство']['spis'].sum()

def spis_gph(row):
    i = i = row['group']
    return rows[i][rows[i]['type'] == 'gph']['spis'].sum()


In [108]:
ans['spis_all'] = ans.apply(lambda row: spis_all(row), axis=1)
ans['spis_osn'] = ans.apply(lambda row: spis_osn(row), axis=1)
ans['spis_vnes'] = ans.apply(lambda row: spis_vnes(row), axis=1)
ans['spis_gph'] = ans.apply(lambda row: spis_gph(row), axis=1)

In [116]:
def fot_all(row):
    i = row['group']
    return rows[i]['sum'].sum() / 1000

def fot_osn(row):
    i = i = row['group']
    return rows[i][rows[i]['type'] == 'Основное место работы']['sum'].sum() / 1000

def fot_vnes(row):
    i = i = row['group']
    return rows[i][rows[i]['type'] == 'Внешнее совместительство']['sum'].sum() / 1000

def fot_gph(row):
    i = i = row['group']
    return rows[i][rows[i]['type'] == 'gph']['sum'].sum() / 1000

In [117]:
ans['fot_all'] = ans.apply(lambda row: fot_all(row), axis=1)
ans['fot_osn'] = ans.apply(lambda row: fot_osn(row), axis=1)
ans['fot_vnes'] = ans.apply(lambda row: fot_vnes(row), axis=1)
ans['fot_gph'] = ans.apply(lambda row: fot_gph(row), axis=1)

In [119]:
for i in ['_osn', '_vnes', '_gph']:
    ans['szp' + i] = ans['fot' + i] / ans['spis' + i] * 1000

In [99]:
rows['all'][rows['all']['type'] == 'gph']['spis'].sum()

0.0

In [85]:
rows['other']['spis'].sum()

9.76923076923077

In [88]:
rows['ped_all'][rows['ped_all']['type'].isna()]['sum'].sum()

0.0